# Embedding Watermark


In [ ]:
from PIL import Image
import numpy as np
import cv2

### Load the host and watermark images


In [ ]:
hr_image = Image.open("host_image.jpg")
wm_image = Image.open("./watermark_image.jpg")

In [ ]:
# Resize the watermark image to the same size as the host image
wm_image_resized = wm_image.resize(hr_image.size)

# Convert both images to grayscale
hr = hr_image .convert("L")
wm = wm_image_resized.convert("L")

In [ ]:
# Convert channels to numpy arrays
hr = np.array(hr)
wm = np.array(wm)
hr,wm

### Function to apply SVD and modify singular values


In [ ]:
# Function to apply SVD and modify singular values
def embed_watermark(hr_channel, wm_channel, alpha=0.6):
    U_hr, sigma_hr, V_hr = np.linalg.svd(hr_channel)
    U_wm, sigma_wm, V_wm = np.linalg.svd(wm_channel)

    sigma_hr_mod = sigma_hr + alpha * sigma_wm
    hr_channel_mod = np.dot(U_hr[:, :sigma_hr_mod.shape[0]], np.dot(
        np.diag(sigma_hr_mod), V_hr[:sigma_hr_mod.shape[0], :]))

    return hr_channel_mod

### Embed watermark


In [ ]:
hr_mod = embed_watermark(hr, wm)

In [ ]:
# Normalize and convert to uint8
hr_mod = cv2.normalize(hr_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
hr_mod

In [ ]:
watermarked_image_grayscale = Image.fromarray(hr_mod)
watermarked_image_grayscale.show()
watermarked_image_grayscale.save('output_image.png')

# Extraction


### Load the host,watermark and watermarked images

In [ ]:
hr_image = Image.open("host_image.jpg")
wm_image = Image.open("./watermark_image.jpg")
watermarked_image = Image.open("./output_image-1.png")

In [ ]:
# Resize the watermark image to the same size as the host image
wm_image_resized = wm_image.resize(hr_image.size)
watermarked_image_resized = watermarked_image.resize(hr_image.size)

# Convert images to RGB (if not already in that mode)
hr_image = hr_image.convert("RGB")
wm_image_resized = wm_image_resized.convert("RGB")
watermarked_image_resized = watermarked_image_resized.convert("RGB")

# Split images into R, G, B channels
hr_r, hr_g, hr_b = hr_image.split()
wm_r, wm_g, wm_b = wm_image_resized.split()
hr_r_mod, hr_g_mod, hr_b_mod = watermarked_image_resized.split()

In [ ]:
# Convert channels to numpy arrays
hr_r = np.array(hr_r)
hr_g = np.array(hr_g)
hr_b = np.array(hr_b)
hr_r, hr_g, hr_b

In [ ]:
wm_r = np.array(wm_r)
wm_g = np.array(wm_g)
wm_b = np.array(wm_b)
wm_r, wm_g, wm_b

In [ ]:
hr_r_mod = np.array(wm_r)
hr_g_mod = np.array(wm_g)
hr_b_mod = np.array(wm_b)
hr_r_mod, hr_g_mod, hr_b_mod

In [ ]:
### Extraction process

In [ ]:
def extract_watermark(hr_mod_channel, hr_channel, wr_channel, alpha=0.6):
    U_hr_mod, sigma_hr_mod, V_hr_mod = np.linalg.svd(
        hr_mod_channel, full_matrices=False)
    U_hr, sigma_hr, V_hr = np.linalg.svd(hr_channel, full_matrices=False)
    U_wm, sigma_wm, V_wm = np.linalg.svd(wr_channel, full_matrices=False)

    sigma_wm_mod = (sigma_hr_mod - sigma_hr) / alpha
    wm_channel_mod = np.dot(U_wm, np.dot(np.diag(sigma_wm_mod), V_wm))

    return wm_channel_mod

### Extract watermark from each channel

In [ ]:
wm_r_mod = extract_watermark(hr_r_mod, hr_r, wm_r)
wm_g_mod = extract_watermark(hr_g_mod, hr_g, wm_g)
wm_b_mod = extract_watermark(hr_b_mod, hr_b, wm_b)

In [ ]:
# Normalize and convert to uint8
wm_r_mod = cv2.normalize(wm_r_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
wm_g_mod = cv2.normalize(wm_g_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
wm_b_mod = cv2.normalize(wm_b_mod, None, 0, 255,
                         cv2.NORM_MINMAX, dtype=cv2.CV_8U)
wm_r_mod, wm_g_mod, wm_b_mod

In [ ]:
# Merge extracted channels back into an RGB image
extracted_watermark = Image.merge("RGB", (Image.fromarray(
    wm_r_mod), Image.fromarray(wm_g_mod), Image.fromarray(wm_b_mod)))
extracted_watermark.save('watermark_output_image-1.png')
extracted_watermark.show()